# City Planner Application

The eventual goal is to have a GUI-based application that can produce a variety of plots that the climate impacts community would find useful.   One of these plots is "Number of Days with Temperature over X Degrees" for some GPS location and time range.

The application should eventually allow choices for:
  * Data source (LENS, CMIP6, NA-CORDEX, etc.)
  * Climate Application  (Extreme Heat Events, Growing Season, Extreme Precipitation Events, etc.)
  * GPS location if applicable
  * Time range
  * Units (Fahrenheit vs. Celsius, etc.)
  * Plot type (Scatter plots vs Box plots, etc.)

In [1]:
# Display output of plots directly in Notebook
%matplotlib inline

In [37]:
import xarray as xr
import numpy as np
import intake
from tqdm.auto import tqdm
import shutil 
import os
from functools import reduce
import pprint
import json
from operator import mul
import random
import yaml
from distributed.utils import format_bytes
import dask
from dask_jobqueue import SLURMCluster
from distributed import Client


In [5]:
dask.config.set({'distributed.dashboard.link': '/proxy/{port}/status'})
dask.config.get('distributed.dashboard')

{'link': '/proxy/{port}/status', 'export-tool': False, 'graph-max-items': 5000}

In [6]:
cluster = SLURMCluster(cores=4, memory="100GB", project="STDD0003")
cluster.adapt(minimum_jobs=2, maximum_jobs=10)
#cluster.scale(jobs=3)
client = Client(cluster)
cluster

### Starting Point:  Create static temperature Plot for Boulder, Colorado from CESM LENS Data

In [7]:

# Load the data catalog.  Use a field separator character for key generation that does not conflict with strings in the catalog.
url = "/glade/collections/cmip/catalog/intake-esm-datastore/catalogs/glade-cesm1-le.json"
col = intake.open_esm_datastore(url, sep='-')

# List a few catalog entries for an idea of the catalog structure.  
# Note: in general, each catalog has different columns.
col.df.head()

,experiment,case,component,stream,variable,date_range,member_id,path,ctrl_branch_year,year_offset
0,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,PRECTMX,04020101-04991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0
1,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,PRECTMX,05000101-05991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0
2,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,PRECTMX,06000101-06991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0
3,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,PRECTMX,07000101-07991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0
4,CTRL,b.e11.B1850C5CN.f09_g16.005,atm,cam.h1,PRECTMX,08000101-08991231,0,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,NaN,1448.0


In [8]:
# Discover what is available for Daily Temperature at Reference Height (TREFHT)
subset = col.search(variable='TREFHT')
subset.unique(columns=['experiment', 'component', 'date_range'])

{'experiment': {'count': 6,
  'values': ['20C', '20C-OIC', 'CTRL', 'CTRL-AMIP', 'CTRL-SLAB-OCN', 'RCP85']},
 'component': {'count': 1, 'values': ['atm']},
 'date_range': {'count': 78,
  'values': ['000101-009912',
   '00010101-00991231',
   '010001-019912',
   '01000101-01991231',
   '010101-019912',
   '01010101-01991231',
   '020001-029912',
   '02000101-02991231',
   '030001-039912',
   '03000101-03991231',
   '040001-049912',
   '04000101-04991231',
   '04020101-04991231',
   '050001-059912',
   '05000101-05991231',
   '060001-069912',
   '06000101-06991231',
   '070001-079912',
   '07000101-07991231',
   '080001-089912',
   '08000101-08991231',
   '090001-099912',
   '090001-100112',
   '09000101-09991231',
   '09000101-10011231',
   '100001-109912',
   '10000101-10991231',
   '110001-119912',
   '11000101-11991231',
   '120001-129912',
   '12000101-12991231',
   '130001-139912',
   '13000101-13991231',
   '140001-149912',
   '14000101-14991231',
   '150001-159912',
   '15000101-1

In [9]:
# Let's limit the date range to [1850, 2100] by choosing a subset of experiments, and choose the "daily" output stream.
subset = col.search(variable='TREFHT', experiment=['HIST', '20C', 'RCP85'], stream='cam.h1')
subset.df

,experiment,case,component,stream,variable,date_range,member_id,path,ctrl_branch_year,year_offset
0,20C,b.e11.B20TRC5CNBDRD.f09_g16.001,atm,cam.h1,TREFHT,18500101-20051231,1,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
1,20C,b.e11.B20TRC5CNBDRD.f09_g16.002,atm,cam.h1,TREFHT,19200101-20051231,2,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
2,20C,b.e11.B20TRC5CNBDRD.f09_g16.003,atm,cam.h1,TREFHT,19200101-20051231,3,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
3,20C,b.e11.B20TRC5CNBDRD.f09_g16.004,atm,cam.h1,TREFHT,19200101-20051231,4,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
4,20C,b.e11.B20TRC5CNBDRD.f09_g16.005,atm,cam.h1,TREFHT,19200101-20051231,5,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
...,...,...,...,...,...,...,...,...,...,...
108,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.101,atm,cam.h1,TREFHT,20060101-21001231,101,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
109,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.102,atm,cam.h1,TREFHT,20060101-21001231,102,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
110,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.103,atm,cam.h1,TREFHT,20060101-21001231,103,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN
111,RCP85,b.e11.BRCP85C5CNBDRD.f09_g16.104,atm,cam.h1,TREFHT,20060101-21001231,104,/glade/collections/cdg/data/cesmLE/CESM-CAM5-B...,402.0,NaN


### Create Xarray Objects from this data subset

Note that the operation that merges data into Xarray Objects is relatively expensive, so we create a Dask cluster to process it quickly.

In [10]:
# Load catalog entries for subset into a dictionary of xarray datasets
dsets = subset.to_dataset_dict(zarr_kwargs={"consolidated": True})
print("\nDataset dictionary keys:\n", dsets.keys())

Dataset(s):   0%|                                        | 0/2 [00:00<?, ?it/s]


--> The keys in the returned dictionary of datasets are constructed as follows:
	'component-experiment-stream'


Dataset(s): 100%|████████████████████████████████| 2/2 [01:37<00:00, 48.59s/it]


Dataset dictionary keys:
 dict_keys(['atm-20C-cam.h1', 'atm-RCP85-cam.h1'])


In [14]:
daily_temps_1850_2005 = dsets['atm-20C-cam.h1']

In [15]:
(boulder_lat, boulder_lon) = (40.01, 244.8)

In [16]:
boulder_temps = daily_temps_1850_2005.sel(lat=boulder_lat, lon=boulder_lon, method='nearest')

In [17]:
boulder_temps_1990 = boulder_temps.sel(time=slice('1990-01-01', '1991-01-01'))

In [18]:
boulder_temps_1990

<xarray.Dataset>
Dimensions:              (cosp_ht: 40, cosp_htmisr: 16, cosp_prs: 7, cosp_scol: 10, cosp_sr: 15, cosp_sza: 5, cosp_tau: 7, cosp_tau_modis: 6, ilev: 31, lev: 30, member_id: 40, nbnd: 2, slat: 256, slon: 288, time: 366)
Coordinates:
  * slon                 (slon) float64 -0.625 0.625 1.875 ... 355.6 356.9 358.1
  * ilev                 (ilev) float64 2.255 5.032 10.16 ... 967.5 985.1 1e+03
  * slat                 (slat) float64 -89.53 -88.59 -87.64 ... 88.59 89.53
  * time                 (time) object 1990-01-01 00:00:00 ... 1991-01-01 00:00:00
    lon                  float64 245.0
    lat                  float64 40.05
  * lev                  (lev) float64 3.643 7.595 14.36 ... 957.5 976.3 992.6
  * cosp_ht              (cosp_ht) float64 240.0 720.0 ... 1.848e+04 1.896e+04
  * cosp_htmisr          (cosp_htmisr) float64 -99.0 0.25 0.75 ... 16.0 58.0
  * cosp_tau             (cosp_tau) float64 0.15 0.8 2.45 6.5 16.2 41.5 219.5
  * cosp_prs             (cosp_prs) float64 900.0 740.0 620.0 ... 245.0 90.0
  * cosp_sr              (cosp_sr) float64 0.605 2.1 4.0 ... 539.5 1.004e+03
  * cosp_scol            (cosp_scol) float64 1.0 2.0 3.0 4.0 ... 8.0 9.0 10.0
  * cosp_tau_modis       (cosp_tau_modis) float64 0.8 2.45 ... 41.5 5.003e+04
  * cosp_sza             (cosp_sza) float64 0.0 15.0 30.0 45.0 60.0
  * member_id            (member_id) int64 1 2 3 4 5 6 ... 101 102 103 104 105
Dimensions without coordinates: nbnd
Data variables:
    cosp_ht_bnds         (time, cosp_ht, nbnd) float64 dask.array<chunksize=(366, 40, 2), meta=np.ndarray>
    f11vmr               (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    nscur                (time) int32 dask.array<chunksize=(366,), meta=np.ndarray>
    sol_tsi              (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    time_written         (time) |S8 dask.array<chunksize=(366,), meta=np.ndarray>
    gw                   (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    ntrk                 (time) int32 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    ndcur                (time) int32 dask.array<chunksize=(366,), meta=np.ndarray>
    hyam                 (time, lev) float64 dask.array<chunksize=(366, 30), meta=np.ndarray>
    mdt                  (time) int32 1800 1800 1800 1800 ... 1800 1800 1800
    nlon                 (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    ntrm                 (time) int32 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    w_stag               (time, slat) float64 dask.array<chunksize=(366, 256), meta=np.ndarray>
    ndbase               (time) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    time_bnds            (time, nbnd) object dask.array<chunksize=(366, 2), meta=np.ndarray>
    hyai                 (time, ilev) float64 dask.array<chunksize=(366, 31), meta=np.ndarray>
    ch4vmr               (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    date_written         (time) |S8 dask.array<chunksize=(366,), meta=np.ndarray>
    cosp_tau_bnds        (time, cosp_tau, nbnd) float64 dask.array<chunksize=(366, 7, 2), meta=np.ndarray>
    cosp_sr_bnds         (time, cosp_sr, nbnd) float64 dask.array<chunksize=(366, 15, 2), meta=np.ndarray>
    f12vmr               (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    cosp_prs_bnds        (time, cosp_prs, nbnd) float64 dask.array<chunksize=(366, 7, 2), meta=np.ndarray>
    nsbase               (time) int32 0 0 0 0 0 0 0 0 0 0 ... 0 0 0 0 0 0 0 0 0
    wnummax              (time) float64 dask.array<chunksize=(366,), meta=np.ndarray>
    ntrn                 (time) int32 1 1 1 1 1 1 1 1 1 1 ... 1 1 1 1 1 1 1 1 1
    hybm                 (time, lev) float64 dask.array<chunksize=(366, 30), meta=np.ndarray>
    nsteph               (time) int32 dask.array<chunksize=(366,), meta=np.ndarray>
    hybi                 (time, ilev) float64 dask.array<chunksize=(366, 31), meta=np.ndarray>
    date                 (time) int32 da

In [20]:
import matplotlib.pyplot as plt



In [29]:
times = boulder_temps_1990.time.data
times[:3]

array([cftime.DatetimeNoLeap(1990-01-01 00:00:00),
       cftime.DatetimeNoLeap(1990-01-02 00:00:00),
       cftime.DatetimeNoLeap(1990-01-03 00:00:00)], dtype=object)

In [40]:
time_indexes = np.arange(1,times.size + 1)

In [33]:
temps = boulder_temps_1990.TREFHT.data.transpose()
temps

dask.array<transpose, shape=(366, 40), dtype=float32, chunksize=(366, 1), chunktype=numpy.ndarray>

In [ ]:
ax = plt.axes()
ax.plot(time_indexes, temps)

In [8]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


### Test NA-CORDEX Catalog Behavior Here

In [ ]:
import intake

In [ ]:
cordex_path = '/glade/u/home/bonnland/GitRepos/PullRequests/intake-esm-datastore-sethmcg/catalogs/glade-na-cordex.json'

In [ ]:
col = intake.open_esm_datastore(cordex_path)

In [ ]:
col

## Explore Zarr files

In [ ]:
import xarray as xr


In [ ]:
ds = xr.open_zarr("/glade/scratch/bonnland/lens-aws/ocn/monthly/cesmLE-CTRL-SFWF.zarr", consolidated=True)

In [ ]:
ds